### Labmda expression review

In [44]:
def square(num):
    return(num**2)

In [45]:
x = 8
square(x)

64

In [46]:
def square(num): return(num**2)

In [47]:
square(42)

1764

In [48]:
x = lambda num: num**2

In [49]:
x(8)

64

In [50]:
even = lambda num: num%2==0

In [51]:
even(9)

False

In [52]:
first = lambda s: s[0]

In [53]:
first('this is a string')

't'

In [54]:
last = lambda x: x[-1]

In [55]:
last('this is a string')

'g'

In [56]:
rev = lambda r: r[::-1]

In [57]:
rev('this is a string')

'gnirts a si siht'

In [58]:
rev('TrentResnor')

'ronseRtnerT'

In [59]:
add = lambda x,y: x+y

In [ ]:
add(3,4)

### Spark and Python

In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext()

In [ ]:
%%writefile example.txt
first line
second line 
third line
forth line
asdfasdf
sadfsafewrwerqw
asdfafetasdfas


In [ ]:
textfile = sc.textFile('example.txt')

In [ ]:
textfile.count()

In [ ]:
textfile.first()

In [ ]:
textfile.first()[0]

In [ ]:
secfilter = textfile.filter(lambda line: 'second' in line)

In [ ]:
secfilter.count()

In [ ]:
secfilter.first().rstrip()

In [ ]:
secfilter.collect()

### RDD Transformations and Actions

In [ ]:
%%writefile example2.txt
first
second line
the third line
then a fourth line

In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
text_rdd = sc.textFile('example2.txt')

In [3]:
words = text_rdd.map(lambda line: line.split())

In [5]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [8]:
allwords = text_rdd.flatMap(lambda l: l.split()).collect()

In [9]:
allwords

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [10]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [11]:
services = sc.textFile('services.txt')

In [12]:
services.take(5)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00']

In [21]:
services.map(lambda x: x.split()).take(5)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00']]

In [19]:
services.map(lambda l: l[1:] if l[0]=='#' else l).collect()

['EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [82]:
services.filter(lambda l: 'CA' in l).collect()

['202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00']

In [83]:
services.map(lambda l: 'CA' in l).collect()

[False, False, False, True, True, False, False]

In [99]:
clean = services.map(lambda l: l[1:] if l[0]=='#' else l)

In [100]:
clean = clean.map(lambda x: x.split())

In [101]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [103]:
clean.map(lambda lst: (lst[3], lst[-1])).collect() # or use (lst[3], lst[5])

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [104]:
pairs = clean.map(lambda lst: (lst[3], lst[-1]))

In [108]:
# Assumes col1 is the key and col2 is the value
rekey = pairs.reduceByKey(lambda amt1, amt2: float(amt1) + float(amt2))

In [109]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [110]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [113]:
# Step 1: Grab (State, Amount) Tuple
step1 = clean.map(lambda x: (x[3], x[-1]))

In [114]:
# Step 2: ReduceByKey 
step2 = step1.reduceByKey(lambda x,y: float(x)+float(y))

In [117]:
# Step 3: Get Rid of State and Amount titles
step3 = step2.filter(lambda t: not t[0]=='State')

In [119]:
# Step 4: Sort Results By Amount
# s[1] look at second value in tuple and sort by that
step4 = step3.sortBy(lambda s: s[1], ascending=False)

In [120]:
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [123]:
x = ["12345", "CA", 123345.42]

In [121]:
def func1(lst):
    return(lst[-1])

In [126]:
def func2(id_st_amt):
    (Id, St, Amt) = id_st_amt
    return(Amt)

In [124]:
func1(x)

123345.42

In [127]:
func2(x)

123345.42